## Overview
    1. Document Loader 
    2. Text splitter
    3. Embedding model
    4. Vector store
    5. Retriever
    6. Prompt template
    7. LLM 
    8. Chain 



In [1]:
import os 
from dotenv import load_dotenv

os.chdir('/Users/reejungkim/Documents/Git/working-in-progress')
load_dotenv()

True

### `1. Document Loader`

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/Users/reejungkim/Documents/Git/HuggingFace/Amazon-2024-Annual-Report.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()
pages = loader.load_and_split()

### 문서 추가 as needed
# loader2 = PyPDFLoader('/Users/reejungkim/Documents/Git/HuggingFace/Amazon-2025-Proxy-Statement.pdf')
# docs2 = loader2.load()
# docs.extend(docs2)

# for page in pages:
#     print(page.metadata, page)

In [3]:
for page in pages[:1]:
    print(page.page_content, page)

ANNUAL REPORT
2 0 2 4 page_content='ANNUAL REPORT
2 0 2 4' metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.0 (Macintosh)', 'creationdate': '2022-02-14T21:08:55-06:00', 'author': 'T&C Composition', 'gts_pdfxconformance': 'PDF/X-1a:2001', 'gts_pdfxversion': 'PDF/X-1:2001', 'keywords': '25-4123-1_2', 'moddate': '2025-04-09T12:45:58-07:00', 'subject': 'Annual Report', 'title': 'Amazon.com, Inc.', 'trapped': '/False', 'source': '/Users/reejungkim/Documents/Git/HuggingFace/Amazon-2024-Annual-Report.pdf', 'total_pages': 91, 'page': 0, 'page_label': '1'}


### `2. Text splitter`

chunk_size 적절한 크기(보통 500~1000자)는 의미 있는 문맥 단위로 자르면서도 내용이 분산되지 않게 합니다. 

chunk_overlap은 텍스트를 쪼개는 과정에서 텍스트의 겹치는 부분을 얼마나 허용할 것인지를 결정합니다. 중요한 단어나 문장이 chunk 경계에 걸쳐 잘릴 수 있음.

- `chunk_size ↓`: semantic sparsity (의미 희박)  
- `chunk_size ↑`: semantic drift (의미 혼합, 벡터 불명확)  
- `chunk_overlap`: 문맥 연속성과 정보 손실 방지 역할


In [4]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)   
texts = text_splitter.split_documents(docs)

# 임베딩

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

model_huggingface = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask'
                                          , model_kwargs = {'device':'cpu'}
                                          , encode_kwargs = {'normalize_embeddings':True})

# 벡터 저장소

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(texts, 
                                   model_huggingface)

In [ ]:
results = vectorstore.similarity_search("How much of the cash used in Operating activities in the year of 2024?, k=5")
#answer: $ 115,877

for r in results:
    print(r.page_content)

# 문서 저장소 ID 확인
vectorstore.index_to_docstore_id


# 저장된 문서의 ID: Document 확인
vectorstore.docstore._dict

In [ ]:
# Retriever: 문서에 포함되어 있는 정보를 검색하고 생성


# Sparse Retriever 
# TF-IDF 또는 BM25와 같은 전통적 정보검색 기법
# 키워드 선택이 검색 품질을 좌우 (간단하고 명확한 키워드 검색에 유리)
from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 4 # set number of documents to retrieve 

# Dense Retriever 
# vector간의 거리 (ex.cosine similarity)
# 키워드가 일치 하지 않아도 의미적으로 관련(뉘앙스와 문맥 일치) 문서 검색
# 복잡한 쿼리 유리
faiss_retriever = vectorstore.as_retriever()


In [ ]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
bm25_retriever.invoke("How much of the cash used in Operating activities in the year of 2024?")


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.prompts import PromptTemplate

template = """<|system|>
You are an assistant for question-answering tasks. 
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you don't know the answer, just say that you don't know. 
Answer in Korean. <|end|>

<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)



### LLM

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint


# 사용할 모델의 저장소 ID를 설정합니다.
repo_id = "microsoft/Phi-3-mini-4k-instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,
    huggingfacehub_api_token=os.environ["huggingface_read"],  # 허깅페이스 토큰
)


In [ ]:

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
# 체인(Chain) 생성
# 단계 8: 체인(Chain) 생성
from langchain_core.runnables import RunnablePassthrough

# def format_docs(docs):
#     """Format retrieved documents into a single string"""
#     return "\n\n".join([f"Document {i+1}: {doc.page_content}" for i, doc in enumerate(docs)])
# 후처리 함수: 여러 문서를 하나의 문자열로 묶는 역할  Format retrieved documents into a single string
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)


# chain = (
#     {
#         "context": lambda x: format_docs(bm25_retriever.invoke(x["question"])),
#         "question": lambda x: x["question"]
#     }
#     | prompt
#     | llm
#     | StrOutputParser()
# )
chain = (
    {"context": faiss_retriever, "question": RunnablePassthrough()}  # 문맥을 검색하고 질문은 그대로 전달
    | prompt  # 문맥 + 질문 → GPT로 전달될 형태로 포맷
    | llm  # GPT가 응답 생성
    | StrOutputParser()  # 출력된 응답을 사람이 읽을 수 있는 텍스트로 정리
)


# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.

# Simpler approach that's easier to debug
def retrieve_and_format(question):
    docs = bm25_retriever.invoke(question)
    return format_docs(docs)

# Test this function first
test_context = retrieve_and_format("what is the revenue of Amazon in 2024")
print("Context:", test_context[:500])

In [ ]:
print(prompt)

### Chroma 기반 문서 벡터화 

from langchain_chroma import Chroma  
db = Chroma.from_documents(texts, model_huggingface )

retriever = Chroma.as_retriever()

question = 'who is Brad D. Smith'
answer = retriever.invoke(question) 


In [ ]:

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)